In [24]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf
from pennylane.templates import RandomLayers
n_epochs = 30
n_qubits=4
PREPROCESS = True
np.random.seed(0)
tf.random.set_seed(0)
dev = qml.device("default.qubit", wires=4)
params = 2 * np.pi * tf.random.uniform([3,50])

In [25]:
def dataset(dataset_no,train_size,test_size):
    n_train =train_size
    n_test =test_size
    if dataset_no==1:
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    elif dataset_no==2:
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
    train_images = x_train[:n_train]
    train_labels = y_train[:n_train]
    test_images = x_test[:n_test]
    test_labels = y_test[:n_test]
    train_images = train_images / 255
    test_images = test_images / 255
    train_images = np.array(train_images[..., tf.newaxis],requires_grad=False)
    test_images = np.array(test_images[..., tf.newaxis],requires_grad=False)
    
    return train_images,test_images,train_labels,test_labels


        
        
        
    

In [26]:
dev = qml.device("default.qubit", wires=4)

In [2]:

@qml.qnode(dev)
def circuit_1(data,params):
    for j in range(n_qubits):
        qml.RY(np.pi * data[j], wires=j)
       
    for i in range(n_qubits):
        qml.RX(params[1,i], wires=i)
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
                
    for i in range(n_qubits):
        qml.RX(params[2,i], wires=i)
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
    
    return [qml.expval(qml.PauliZ(w)) for w in range(4)]

    
    
        
        

In [3]:
@qml.qnode(dev)
def circuit_2(data,params):
    for j in range(n_qubits):
            qml.RX(np.pi * data[j], wires=j)
           
    for j in range(n_qubits):
            qml.Rot(params[1,3*j],params[1,3*j+1],params[1,3*j+2], wires=j)
    for j in range(0, n_qubits):
                if j == 0:
                   for i in range(0, n_qubits - 1):
                       qml.CZ(wires=[j, i + 1])
                elif j == 1:
                     qml.CZ(wires=[j, j - 1])
                     for i in range(0, n_qubits - 2):
                         qml.CZ(wires=[j, i + 2])
                
                elif j == 2:
                     qml.CZ(wires=[j, j + 1])
                     for i in range(n_qubits - 1, 1, -1):
                         qml.CZ(wires=[j, i - 2])
                elif j == 3:
                     for i in range(n_qubits - 1, 0, -1):
                         qml.CZ(wires=[j, i - 1])
                        
                        
    for j in range(n_qubits):
            qml.Rot(params[2,3*j],params[2,3*j+1],params[2,3*j+2], wires=j)
            
    for j in range(0, n_qubits):
                
                if j == 0:
                   for i in range(0, n_qubits - 1):
                       qml.CZ(wires=[j, i + 1])
                elif j == 1:
                     qml.CZ(wires=[j, j - 1])
                     for i in range(0, n_qubits - 2):
                         qml.CZ(wires=[j, i + 2])
                
                elif j == 2:
                     qml.CZ(wires=[j, j + 1])
                     for i in range(n_qubits - 1, 1, -1):
                         qml.CZ(wires=[j, i - 2])
                elif j == 3:
                     for i in range(n_qubits - 1, 0, -1):
                         qml.CZ(wires=[j, i - 1])

    return [qml.expval(qml.PauliZ(w)) for w in range(4)]




In [4]:

@qml.qnode(dev)
def circuit_3(data,params):
    for i in range(0, n_qubits):
        qml.RY(np.pi * data[i], wires=i)
          
    for i in range(0, n_qubits):
        qml.RX(params[1,i], wires=i)
        qml.RZ(params[1,i], wires=i)
    qml.CRX(params[1,n_qubits], wires=[n_qubits - 1, 0])
    for i in range(n_qubits - 1, 0, -1):
        qml.CRX(params[1,n_qubits+i], wires=[i - 1, i])
        
    for i in range(0, n_qubits):
        qml.RX(params[2,i], wires=i)
        qml.RZ(params[2,i], wires=i)
    qml.CRX(params[2,n_qubits], wires=[n_qubits - 1, 0])
    for i in range(n_qubits - 1, 0, -1):
        qml.CRX(params[2,n_qubits+i], wires=[i - 1, i])
        
    return [qml.expval(qml.PauliZ(w)) for w in range(4)]

In [5]:

@qml.qnode(dev)
def circuit_4(data,params):
    for i in range(0, n_qubits):
        qml.RY(np.pi * data[i], wires=i)
           
             
    for i in range(0, n_qubits):
        qml.RX(params[1,i], wires=i)
        qml.RZ(params[1,i], wires=i)

    for i in range(0,n_qubits - 1,2):
        qml.CRX(params[1,i], wires=[i, i+1])

    for i in range(0, n_qubits):
        qml.RX(params[1,i], wires=i)
        qml.RZ(params[1,i], wires=i)

    for i in range(1,n_qubits - 1,2):
        qml.CRX(params[1,i], wires=[i, i+1])
                    
    for i in range(0, n_qubits):
        qml.RX(params[2,i], wires=i)
        qml.RZ(params[2,i], wires=i)

    for i in range(0,n_qubits - 1,2):
        qml.CRX(params[2,i], wires=[i, i+1])

    for i in range(0, n_qubits):
        qml.RX(params[2,i], wires=i)
        qml.RZ(params[2,i], wires=i)

    for i in range(1,n_qubits - 1,2):
        qml.CRX(params[2,i], wires=[i, i+1])
        
    return [qml.expval(qml.PauliZ(w)) for w in range(4)]
        

In [6]:

@qml.qnode(dev)
def circuit_5(data,params):
    for j in range(n_qubits):
        qml.RY(np.pi*data[j], wires=j)
            
    for j in range(0, n_qubits):
        qml.U3(params[1,j], params[1,j + 1], params[1,j + 2], wires=j)
    for i in range(0, n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
    for j in range(0, int(n_qubits / 2)):
        qml.RY(params[1,j], wires=2 * j)
        qml.RZ(params[1,j], wires=2 * j + 1)
    for i in range(0, n_qubits - 1):
        qml.CNOT(wires=[i,i+1])
    for j in range(0, int(n_qubits / 2)):
        qml.RY(params[1,j], wires=2 * j)
    for i in range(0, n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
    for j in range(0, n_qubits):
        qml.U3(params[2,j], params[2,j + 1], params[2,j + 2], wires=j)
        
    return [qml.expval(qml.PauliZ(w)) for w in range(4)]

In [7]:

@qml.qnode(dev)
def circuit_6(data,params):
    for j in range(n_qubits):
        qml.RY(np.pi*data[j], wires=j)
        
    for idx in range(n_qubits):
        qml.Hadamard(wires=idx)
    for i in range(n_qubits-1,0,-1):
        qml.CZ(wires=[i,i-1])
    for i in range(0, n_qubits):
        qml.RX(params[1,i], wires=i)
                
                
    for idx in range(n_qubits):
        qml.Hadamard(wires=idx)
    for i in range(n_qubits-1,0,-1):
        qml.CZ(wires=[i,i-1])
    for i in range(0, n_qubits):
        qml.RX(params[2,i], wires=i)
        
    return [qml.expval(qml.PauliZ(w)) for w in range(4)]
                 

In [14]:
def quanv(image,circuit_no):
    """Convolves the input image with many applications of the same quantum circuit."""
    #params =2*np.pi*tf.random.uniform([3,20])
    #params=2 * np.pi * tf.random.uniform([20])
    out = np.zeros((14, 14, 4))
    if circuit_no==1:
        for j in range(0, 28, 2):
            for k in range(0, 28, 2):
                q_results = circuit_1(
                [
                    image[j, k, 0],
                    image[j, k + 1, 0],
                    image[j + 1, k, 0],
                    image[j + 1, k + 1, 0]
                ],params)
                for i in range(4):
                    out[j // 2, k // 2, i] = q_results[i]
    
    elif circuit_no==2:
         for j in range(0, 28, 2):
            for k in range(0, 28, 2):
                q_results = circuit_2(
                [
                    image[j, k, 0],
                    image[j, k + 1, 0],
                    image[j + 1, k, 0],
                    image[j + 1, k + 1, 0]
                ],params)
                for i in range(4):
                    out[j // 2, k // 2, i] = q_results[i]
                    
    elif circuit_no==3:
        
         for j in range(0, 28, 2):
            for k in range(0, 28, 2):
                q_results = circuit_3(
                [
                    image[j, k, 0],
                    image[j, k + 1, 0],
                    image[j + 1, k, 0],
                    image[j + 1, k + 1, 0]
                ],params)
                for i in range(4):
                    out[j // 2, k // 2, i] = q_results[i]
                    
    elif circuit_no==4:
         
        for j in range(0, 28, 2):
            for k in range(0, 28, 2):
                q_results = circuit_4(
                [
                    image[j, k, 0],
                    image[j, k + 1, 0],
                    image[j + 1, k, 0],
                    image[j + 1, k + 1, 0]
                ],params)
                for i in range(4):
                    out[j // 2, k // 2, i] = q_results[i]
                    
    elif circuit_no==5:
        
        for j in range(0, 28, 2):
            for k in range(0, 28, 2):
                q_results = circuit_5(
                [
                    image[j, k, 0],
                    image[j, k + 1, 0],
                    image[j + 1, k, 0],
                    image[j + 1, k + 1, 0]
                ],params)
                
                for i in range(4):
                    out[j // 2, k // 2, i] = q_results[i]
                    
                    
    elif circuit_no==6:
        for j in range(0, 28, 2):
            for k in range(0, 28, 2):
                q_results = circuit_6(
                [
                    image[j, k, 0],
                    image[j, k + 1, 0],
                    image[j + 1, k, 0],
                    image[j + 1, k + 1, 0]
                ],params)
                
                for i in range(4):
                    out[j // 2, k // 2, i] = q_results[i]
   

    return out
  

In [27]:
train_images,test_images,train_labels,test_labels=dataset(2,250,60)


In [28]:
if PREPROCESS == True:
    q_train_images = []
    print("Quantum pre-processing of train images:")
    for idx, img in enumerate(train_images):
        print("{}/{}        ".format(idx + 1, n_train), end="\r")
        q_train_images.append(quanv(img,3)) #please select the circuit number
    q_train_images = np.asarray(q_train_images)

    q_test_images = []
    print("\nQuantum pre-processing of test images:")
    for idx, img in enumerate(test_images):
        print("{}/{}        ".format(idx + 1, n_test), end="\r")
        q_test_images.append(quanv(img,3))
    q_test_images = np.asarray(q_test_images)



Quantum pre-processing of train images:
250/250        
Quantum pre-processing of test images:


In [29]:


model = tf.keras.models.Sequential([
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(1, 1), padding='same'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(10, activation="softmax")
    ])

opt=tf.keras.optimizers.Adam(learning_rate=0.01)


model.compile(
        optimizer=opt,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],)



In [30]:

model1 = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=4, kernel_size=(2,2),input_shape=(28,28,1),activation='relu'),
        tf.keras.layers.Conv2D(filters=4, kernel_size=(2,2),activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(1, 1), padding='same'),
        #tf.keras.layers.Conv2D(filters=4, kernel_size=(2,2),activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(10, activation="softmax")
    ])
opt=tf.keras.optimizers.Adam(learning_rate=0.01)
model1.compile(
        optimizer=opt,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],)

In [31]:

q_history = model.fit(q_train_images,train_labels,validation_data=(q_test_images, test_labels),batch_size=16,epochs=n_epochs,verbose=2)

Epoch 1/30
16/16 - 0s - loss: 2.4658 - accuracy: 0.2600 - val_loss: 1.7890 - val_accuracy: 0.2333
Epoch 2/30
16/16 - 0s - loss: 1.6815 - accuracy: 0.3320 - val_loss: 1.4942 - val_accuracy: 0.3667
Epoch 3/30
16/16 - 0s - loss: 1.5201 - accuracy: 0.4040 - val_loss: 1.3939 - val_accuracy: 0.4500
Epoch 4/30
16/16 - 0s - loss: 1.4457 - accuracy: 0.4120 - val_loss: 1.2279 - val_accuracy: 0.5333
Epoch 5/30
16/16 - 0s - loss: 1.2920 - accuracy: 0.4840 - val_loss: 1.2298 - val_accuracy: 0.5667
Epoch 6/30
16/16 - 0s - loss: 1.2264 - accuracy: 0.5040 - val_loss: 1.1092 - val_accuracy: 0.6333
Epoch 7/30
16/16 - 0s - loss: 1.1391 - accuracy: 0.5200 - val_loss: 1.1534 - val_accuracy: 0.6000
Epoch 8/30
16/16 - 0s - loss: 1.1209 - accuracy: 0.5800 - val_loss: 1.1295 - val_accuracy: 0.5833
Epoch 9/30
16/16 - 0s - loss: 0.9595 - accuracy: 0.6400 - val_loss: 1.1964 - val_accuracy: 0.5833
Epoch 10/30
16/16 - 0s - loss: 1.0232 - accuracy: 0.5720 - val_loss: 1.1129 - val_accuracy: 0.6500
Epoch 11/30
16/16 -

In [32]:

c_history = model1.fit(train_images,train_labels,validation_data=(test_images, test_labels),batch_size=16,epochs=n_epochs,verbose=2)

Epoch 1/30
16/16 - 0s - loss: 1.6961 - accuracy: 0.4240 - val_loss: 1.1792 - val_accuracy: 0.5333
Epoch 2/30
16/16 - 0s - loss: 0.9216 - accuracy: 0.6840 - val_loss: 1.2475 - val_accuracy: 0.5667
Epoch 3/30
16/16 - 0s - loss: 0.6748 - accuracy: 0.7880 - val_loss: 1.2351 - val_accuracy: 0.6333
Epoch 4/30
16/16 - 0s - loss: 0.5019 - accuracy: 0.8120 - val_loss: 1.1169 - val_accuracy: 0.6500
Epoch 5/30
16/16 - 0s - loss: 0.4128 - accuracy: 0.8680 - val_loss: 1.4817 - val_accuracy: 0.6667
Epoch 6/30
16/16 - 0s - loss: 0.3277 - accuracy: 0.8920 - val_loss: 1.8808 - val_accuracy: 0.6000
Epoch 7/30
16/16 - 0s - loss: 0.4393 - accuracy: 0.8640 - val_loss: 1.3069 - val_accuracy: 0.6167
Epoch 8/30
16/16 - 0s - loss: 0.2057 - accuracy: 0.9280 - val_loss: 1.9713 - val_accuracy: 0.6500
Epoch 9/30
16/16 - 0s - loss: 0.2460 - accuracy: 0.9120 - val_loss: 1.5077 - val_accuracy: 0.7167
Epoch 10/30
16/16 - 0s - loss: 0.1693 - accuracy: 0.9440 - val_loss: 1.8703 - val_accuracy: 0.6833
Epoch 11/30
16/16 -